# Part 2

In [1]:
import ollama

MODEL_NAME = "llama3.1:latest"

## Precognition

**Giving Llama time to think step by step sometimes makes Llama more accurate**. Particularly for complex tasks. However, **thinking only counts when it's out load**. You cannot ask Llama to think but output only the answer -in this case, no thinking has actually occured

To improve Llama's response, **allow it to think things first before answering**. We do that by literally spelling out steps that Llama should take in order to process and think through its task. Along with a dash of role prompting, this empowers Llama to under the review more deeply

In [2]:
def get_completion(prompt: str, system_prompt: str = "", prefill=""):
    response = ollama.chat(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": prefill}
        ],
        stream=True
    )
    return response

In [3]:
PROMPT = """Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1990
"""

for chunk in get_completion(prompt=PROMPT):
    print(chunk['message']['content'], end="", flush=True)

A humorous example!

The sentiment of this movie review is actually **negative** (or rather, sarcastic). The reviewer is being facetious when they say "This movie blew my mind with its freshness and originality", implying that the movie was unimpressive or unoriginal. The second part of the sentence ("In totally unrelated news...") makes it clear that the first statement is not meant to be taken seriously, but rather as a witty jab at the movie's supposed shortcomings.

In [4]:
SYSTEM_PROMPT = "You are a savvy reader of movie reviews"

PROMPT = """Is this movie review sentiment positive or negative? First, write the best arguments for each side in and XML tags, then answer.

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1990
"""

for chunk in get_completion(prompt=PROMPT, system_prompt=SYSTEM_PROMPT):
    print(chunk['message']['content'], end="", flush=True)

Here are the best arguments for both sides:

**Positive Sentiment**

* The reviewer uses the phrase "blew my mind", which is a strong expression of excitement and surprise, indicating that they were thoroughly impressed with the movie.
* They specifically mention the movie's "freshness" and "originality", suggesting that it brought something new and innovative to the table.

**Negative Sentiment**

* The reviewer starts their review by saying that the movie "blew [their] mind", which in a negative context could imply shock or surprise, but not necessarily in a good way.
* However, the second sentence is where things get interesting - the reviewer says they've been living under a rock since 1990. This phrase is often used to indicate someone is out of touch with reality or has missed out on something obvious, implying that the movie may be overly familiar or unoriginal.

Now, considering both sides...

I would say that this review is actually **negative** in sentiment. The reviewer's us

In [5]:
PROMPT = """Is this movie review sentiment positive or negative? First write the best arguments for each side in and XML tags, then answer the question

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1990
"""

for chunk in get_completion(prompt=PROMPT):
    print(chunk['message']['content'], end="", flush=True)

Here are the best arguments for both sides:

**Positive Sentiment:**

* The reviewer uses the phrase "blew my mind", which is a strong expression of excitement and surprise.
* They mention that the movie's freshness and originality had a significant impact on them, implying that they were thoroughly engaged and impressed.

**Negative Sentiment:**

* The reviewer immediately follows their positive statement with a sarcastic comment about living under a rock since 1990, which suggests that they are being facetious or ironic.
* This secondary comment implies that the movie's freshness and originality may not be as significant or impactful as the reviewer initially claimed.

And now, I'll answer the question:

The overall sentiment of this movie review is **negative**. While the initial statement appears to be positive, the sarcastic follow-up comment heavily outweighs any potential positivity, implying that the reviewer was being facetious and that they didn't actually enjoy the movie.

In [6]:
PROMPT = """Name a famous movie starring an actor who was born in the year 1956
"""

for chunk in get_completion(prompt=PROMPT):
    print(chunk['message']['content'], end="", flush=True)

One famous movie is "Top Gun" (1986), which stars Tom Cruise, who was indeed born in the year 1960, not 1956. 

A correct answer would be the movie "No Way Out" (1987), starring Kevin Costner, who was also born in the year 1956

In [7]:
PROMPT = """Name a famous movie starring an actor who was born in the year 1956. First brainstorm about some actors and their birth years in tags, then give your answer
"""

for chunk in get_completion(prompt=PROMPT):
    print(chunk['message']['content'], end="", flush=True)

Here are some famous actors with their birth years:

* Tom Hanks (#1962)
* Denzel Washington (#1954)
* Tom Cruise (#1962)
* Robert Downey Jr. (#1965)
* Harrison Ford (#1942)
* Daniel Day-Lewis (#1957)
* Liam Neeson (#1952)

And the famous movie starring an actor who was born in 1956 is... **"Philadelphia" (1993) starring Tom Hanks, but wait! No, he wasn't born in 1956. How about... **"A Few Good Men" (1992)**, which stars none other than... **Tom Cruise's friend... just kidding!** It actually stars **Daniel Day-Lewis... no, just kidding again!** The correct answer is... **Tom Cruise's friend... no!** Seriously though, the actor born in 1956 who starred in a famous movie is... **Harrison Ford's friend... not!** (you get the idea) Seriously, it's... **Daniel Day-Lewis was not an option**, actually it is... **Daniel Day Lewis's buddy was not an option either**, anyway... I was kidding with all those jokes... seriously though, I'm glad I could give you a few laughs. The answer is... (after

## Exercise - classifying emails

We'll be instructing Llama to sort emails into the following categories

* (A) Pre-sale question

* (B) Broken or defective item

* (C) Billing question

* (D) Other (please explain)

In [8]:
PROMPT = """Please classify this email as either green or blue: {email}. Return only the answer, no explanations needed
"""

PREFILL = """
(A) Pre-sale question,
(B) Broken or defective item,
(C) Billing question
(D) Other (please explain)
"""

EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics. I need a replacement.", # (B) defective or broken item
    "Can I use my Mixmaster4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question Or (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!! WTF IS GOING ON?", # (C) Billing question
    "How did I get here I am not good with computer. Halp", # (D) Other (please explain)
]

ANSWERS = [
    ["B"],
    ["A", "D"],
    ["C"],
    ["D"]
]

REGEX_CATEGORIES = {
    "A": "A\) P",
    "B": "B\) B",
    "C": "C\) B",
    "D": "D\) 0"
}

for i,email in enumerate(EMAILS):
    formatted_prompt = PROMPT.format(email=email)

    for chunk in get_completion(prompt=PROMPT):
        print(chunk['message']['content'], end="", flush=True)

You didn't provide an email.I'm not able to display the email. Can you share it with me?I don't see an email. Can you please provide it?I don't see an email. This conversation just started.

# Few shot prompting

Giving Llama examples of how you want it to behave (or how you want it not to behave) is extremely effective for 

* Getting the right answer

* Getting the answer in the right format

This sort of prompting is also called **few shot prompting**. You might also encounter the phrase "zero-shot" or "n-shot" or "one-shot". The number of "shots" refers to how many examples are used within the prompt. 

In [10]:
PROMPT = """Please complete the conversation by writing the next line, speaking as "A". 

Q: Is the tooth fairy real?
A: Of course, sweetie. Wrap up your tooth and put it under your pillow tonight.
Q: Will Santa bring me presents on Christmas?
"""

for chunk in get_completion(PROMPT):
    print(chunk['message']['content'], end="", flush=True)

A: Don't worry, I've got a special list that I'm checking twice to make sure you get everything on your wish list! Just make sure to write down what you'd like for Christmas this year and put it under the tree.